In [5]:
import numpy as np
import pandas as pd
import glob
import math 
import os
import lvm_read
import pickle
from datetime import datetime, timedelta
import xarray as xr

In [6]:
#user inputs
#Absolute directory of the folders containing LVM and GPS Data
lvmDirectory = "C:\\Github Repositories\\LabView-Garmin-To-NETCDF\\Test Files 2\\LVM Files"
gpsDirectory = "C:\\Github Repositories\\LabView-Garmin-To-NETCDF\\Test Files 2\\GPS Files"

#convert fit files from GPS to CSVs using the following website:
#https://www.fitfileviewer.com/

In [7]:
#obtain and store GPS data
#https://www.geeksforgeeks.org/how-to-merge-multiple-csv-files-into-a-single-pandas-dataframe/
#merge the files
joined_files = os.path.join(gpsDirectory, "*.csv")

#list of all joined files returned
joined_list = glob.glob(joined_files)

#All files joined to get nontimestamp converted GPS Data
gpsData = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

#Garmin has GPS data starting at a time of Sunday, Dec 31st, 1989. To convert to a time that is useable, first convert to epoch time by adding 631065600 seconds. Then convert epoch time to now time.
#http://www.nlvocables.com/blog/?p=969
gpsData.timestamp = gpsData.timestamp.astype(int) + 631065600

#convert to regular date time
gpsData['timestamp'] = pd.to_datetime(gpsData['timestamp'], unit = 's')

#convert GPS data to degrees
#https://gis.stackexchange.com/questions/371656/garmin-fit-coordinate-system
gpsData['position_lat'] = gpsData.position_lat / 11930465
gpsData['position_long'] = gpsData.position_long / 11930465

gpsData

#note, for whatever reason, the first file: "2022-10-19 9.56.49_record_1.csv" displays the wrong date for some reason I don't understand. The forumla works for every other file

,timestamp,position_lat,position_long,enhanced_speed,enhanced_altitude
0,2022-10-17 23:09:35,47.650750,-122.312664,1.53,37.4
1,2022-10-17 23:09:42,47.650520,-122.312864,1.10,37.4
2,2022-10-17 23:10:00,47.650052,-122.312890,0.47,37.4
3,2022-10-17 23:10:27,47.649865,-122.312869,0.41,37.4
4,2022-10-17 23:10:48,47.649726,-122.312897,0.11,37.4
...,...,...,...,...,...
1374,2022-10-28 17:42:56,48.123788,-122.495446,2.18,37.4
1375,2022-10-28 17:43:19,48.123883,-122.495149,0.68,37.4
1376,2022-10-28 17:43:44,48.123853,-122.495134,1.06,37.4
1377,2022-10-28 17:43:46,48.123839,-122.495155,1.10,37.4


In [9]:
#obtain lvm data
##code from this: https://github.com/openmodal/lvm_read/blob/master/Showcase%20lvm_read.ipynb
os.chdir(lvmDirectory)

lvmData = []

#stores all LVM data into a list of files
for file in os.listdir(lvmDirectory):
    lvm = lvm_read.read(file, read_from_pickle= False, dump_file = False)
    #print(lvm)
    lvmData.append(lvm[0])

#change to pandas
lvmDataPD = pd.DataFrame(lvmData)

In [10]:
#code plan
#outer loop is going through all data files and appending it to the end of 2 arrays: timeStamp & voltage
#obtain start time value, convert to a date time object.
#grab # of samples and use it for inner loop
#grab time delta and use it to add time delta to timestamp
#in same inner loop, grab voltages while at it
#append both values to a time and voltage array
#convert to dictionary and then convert to Pandas DF

timeArray = []
voltageArray = []

for i in range(len(lvmDataPD.index)):
    #grab time
    dateConvertChar = lvmDataPD['Date'][i][0].replace('/','-')
    time = dateConvertChar + ' ' + lvmDataPD['Time'][i][0]

    #truncates time string so it can be converted to a dateTime obj
    time = time[0:26]
    dateTime = datetime.strptime(time,'%Y-%m-%d %H:%M:%S.%f')

    #uses number of samples to obtain time deltas
    #Also adds voltages to a seperate array as well
    for j in range(lvmDataPD['Samples'][i][0]):
        #time
        timeArray.append(dateTime)
        dateTime = dateTime + timedelta(microseconds=500)

        #voltages
        #i is for dataset
        #j is for going through each voltage within the dataset
        #0 at end is to ensure it is not an array from the LVM file
        voltage = lvmDataPD['data'][i][j][0]
        voltageArray.append(voltage)
        
#convert to dictionaries for easy conversion to PD dataframe
d = {'Time': timeArray, 'Voltage': voltageArray}
voltagesDF = pd.DataFrame(d)

voltagesDF

,Time,Voltage
0,2022-10-19 19:38:25.973860,2.996339
1,2022-10-19 19:38:25.974360,2.981532
2,2022-10-19 19:38:25.974860,3.149671
3,2022-10-19 19:38:25.975360,3.354005
4,2022-10-19 19:38:25.975860,3.486608
...,...,...
259995,2022-10-19 19:40:54.982697,0.319930
259996,2022-10-19 19:40:54.983197,0.277813
259997,2022-10-19 19:40:54.983697,0.239644
259998,2022-10-19 19:40:54.984197,0.206740


Code plane V2 w/ linear interpolation 12/14/22

1. set arrays for 
    - position_lat
    - position_long
    - enhanced_speed
    - enhanced 
    
    Also get GPS dataset length
    - gpsLength

    Also set starting value to 0
    - jstart = 0

2. Outer Loop
    - Get time from voltage Data Array (i)

3. Inner Loop
    obtain the following values
    - gps Timestamp 1 (j)
    - gps Timestamp 2 (j+1)

    Check time is inbetween those two times
    - Timestamp 1 <= Time <= Timestamp 2
        - if it is between those two times
        - set jstart = j
        
        Linear interpolation (use point slope):
        - find interpolated:
            1. position_lat
            2. position_long
            3. enhanced_speed
            4. enhanced_altitude
        
        Append the following to each array
            1. position_lat
            2. position_long
            3. enhanced_speed
            4. enhanced_altitude

4. Outside the outer loop
    - combine arrays into a dictionary
    - convert dictionaries into a pandas dataframe
    - merge this new dataframe with voltage dataframe

In [24]:
#setup arrays
position_lat = []
position_long = []
enhanced_speed = []
enhanced_altitude =[]

#voltageDF length
timeLength = len(gpsData.index)

#starting j
jstart = 0

#GPS coords code
for i in range(len(voltagesDF.index)):
    #get time
    time = voltagesDF['Time'][i]

    #time as int
    timeInt = int(time.strftime('%Y%m%d%H%M%S%f'))

    for j in range(jstart, timeLength-1):

        #set jstart to current j
        jstart = j
        
        #GPS times
        gpsTimeStamp1 = gpsData['timestamp'][j]
        gpsTimeStamp2 = gpsData['timestamp'][j+1]

        #convert GPS timestamps to ints for linear interp
        gpsTimeStamp1Int = int(gpsTimeStamp1.strftime('%Y%m%d%H%M%S%f'))
        gpsTimeStamp2Int = int(gpsTimeStamp2.strftime('%Y%m%d%H%M%S%f'))

        #position_lat
        gps_lat1 = gpsData['position_lat'][j]
        gps_lat2 = gpsData['position_lat'][j+1]

        #position_long
        gps_long1 = gpsData['position_long'][j]
        gps_long2 = gpsData['position_long'][j+1]

        #enhanced_speed
        gps_speed1 = gpsData['enhanced_speed'][j]
        gps_speed2 = gpsData['enhanced_speed'][j+1]

        #enhanced_altitude
        gps_alt1 = gpsData['enhanced_altitude'][j]
        gps_alt2 = gpsData['enhanced_altitude'][j+1]

        if (gpsTimeStamp1 <= time <= gpsTimeStamp2):
            #linear interp:
            #position_lat linear interp + add to array
            latSlope = (gps_lat2 - gps_lat1) / (gpsTimeStamp2Int - gpsTimeStamp1Int)
            interp_Position_Lat = latSlope*(timeInt - gpsTimeStamp1Int) + gps_lat1
            position_lat.append(interp_Position_Lat)

            #position_long linear interp + add to array
            longSlope = (gps_long2 - gps_long1) / (gpsTimeStamp2Int - gpsTimeStamp1Int)
            interp_Position_Long = longSlope*(timeInt - gpsTimeStamp1Int) + gps_long1
            position_long.append(interp_Position_Long)

            #enhanced_speed linear interp + add to array
            speedSlope = (gps_speed2 - gps_speed1) / (gpsTimeStamp2Int - gpsTimeStamp1Int)
            interp_Position_Speed = speedSlope*(timeInt - gpsTimeStamp1Int) + gps_speed1
            enhanced_speed.append(interp_Position_Speed)

            #enhanced_alt linear interp + add to array
            altSlope = (gps_alt2 - gps_alt1) / (gpsTimeStamp2Int - gpsTimeStamp1Int)
            interp_Position_Alt = altSlope*(timeInt - gpsTimeStamp1Int) + gps_alt1
            enhanced_altitude.append(interp_Position_Alt)

            break
           
#convert to dictionaries for easy conversion to PD dataframe
d1 = {'position_lat': position_lat, 'position_long': position_long, 'enhanced_speed': enhanced_speed, 'enhanced_altitude': enhanced_altitude}
GPS = pd.DataFrame(d1)

GPS

,position_lat,position_long,enhanced_speed,enhanced_altitude
0,48.132151,-122.525603,0.346043,37.6
1,48.132151,-122.525603,0.346043,37.6
2,48.132151,-122.525603,0.346042,37.6
3,48.132151,-122.525603,0.346042,37.6
4,48.132151,-122.525603,0.346042,37.6
...,...,...,...,...
259995,48.131766,-122.525632,0.329422,37.6
259996,48.131766,-122.525632,0.329422,37.6
259997,48.131766,-122.525632,0.329422,37.6
259998,48.131766,-122.525632,0.329423,37.6


In [25]:
#mergeDataFrame together
HTI_DATA = pd.concat([voltagesDF,GPS], axis=1)
HTI_DATA

,Time,Voltage,position_lat,position_long,enhanced_speed,enhanced_altitude
0,2022-10-19 19:38:25.973860,2.996339,48.132151,-122.525603,0.346043,37.6
1,2022-10-19 19:38:25.974360,2.981532,48.132151,-122.525603,0.346043,37.6
2,2022-10-19 19:38:25.974860,3.149671,48.132151,-122.525603,0.346042,37.6
3,2022-10-19 19:38:25.975360,3.354005,48.132151,-122.525603,0.346042,37.6
4,2022-10-19 19:38:25.975860,3.486608,48.132151,-122.525603,0.346042,37.6
...,...,...,...,...,...,...
259995,2022-10-19 19:40:54.982697,0.319930,48.131766,-122.525632,0.329422,37.6
259996,2022-10-19 19:40:54.983197,0.277813,48.131766,-122.525632,0.329422,37.6
259997,2022-10-19 19:40:54.983697,0.239644,48.131766,-122.525632,0.329422,37.6
259998,2022-10-19 19:40:54.984197,0.206740,48.131766,-122.525632,0.329423,37.6


12/15/22
TO-DO
- Use with more data and splice each into each day
- Convert pandas dataframe from each day into a netCDF file